In [19]:
import numpy as np
from collections import defaultdict
import pandas as pd
import pickle
import nltk
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import sklearn_crfsuite
from sklearn_crfsuite import CRF
from sklearn_crfsuite.metrics import flat_classification_report


In [11]:
df = pd.read_csv("/Users/indusha-22955/Documents/Learning/NER dataset.csv",encoding="latin1")
df.ffill(axis=0, inplace=True)
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O


In [ ]:
agg_func = lambda s: [
(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                s["POS"].values.tolist(),
                                s["Tag"].values.tolist())
]
#Zip working

"""
a = ("John", "Charles", "Mike","uyt","pops")
b = ("Jenny", "Christy", "Monica")

x = zip(a, b)
#(('John', 'Jenny'), ('Charles', 'Christy'), ('Mike', 'Monica'))
"""

grouped = df.groupby("Sentence #").apply(agg_func) 
print(type(grouped))
sentences = [s for s in grouped]

<class 'pandas.core.series.Series'>


/var/folders/8s/n5hd259j209817bmcvswgr4m0000gp/T/ipykernel_63436/648392778.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = df.groupby("Sentence #").apply(agg_func)


In [14]:
words_all = [w[0].lower() for s in sentences for w in s]
print(len(words_all))
words = list(set(words_all))
print(len(words))

words.append("ENDPAD")

tags = list(set([w[2] for s in sentences for w in s]))

word2idx = {w: i for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}
idx2word = {i: w for w, i in word2idx.items()}
idx2tag = {i: t for t, i in tag2idx.items()}

print(len(word2idx),len(tag2idx))

1048575
31817
31818 17


https://medium.com/towards-data-science/implementing-a-linear-chain-conditional-random-field-crf-in-pytorch-16b0b9c4b4ea

In [ ]:
def word2features(sent, i):
    """Extracts features for a given word in a sentence."""
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],  # Suffix
        'word[-2:]': word[-2:],  # Suffix
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True  # Beginning of Sentence

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True  # End of Sentence

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    """Extracts labels (NER tags) for an entire sentence."""
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    """Extracts tokens (words) for an entire sentence."""
    return [token for token, postag, label in sent]


def prepare_crf_data(sentences):
    X = [sent2features(s) for s in sentences]
    y = [sent2labels(s) for s in sentences]
    return X, y


def train_crf(X_train, y_train):
    """Trains the CRF model."""
    crf = CRF(algorithm='lbfgs', #Limited-memory BFGS (L-BFGS or LM-BFGS) 
              c1=0.1,
              c2=0.1,
              max_iterations=100,
              all_possible_transitions=True)
    crf.fit(X_train, y_train)
    return crf

def evaluate_crf(crf, X_test, y_test):
    y_pred = crf.predict(X_test)
    report = flat_classification_report(y_test, y_pred)
    print(report)
    return y_pred


In [21]:
train_sentences, test_sentences = train_test_split(sentences, test_size=0.2, random_state=42)


X_train, y_train = prepare_crf_data(train_sentences)
X_test, y_test = prepare_crf_data(test_sentences)

In [22]:
crf = train_crf(X_train, y_train)

In [24]:
y_pred = evaluate_crf(crf, X_test,y_test)

              precision    recall  f1-score   support

       B-art       0.36      0.13      0.19        94
       B-eve       0.56      0.36      0.43        70
       B-geo       0.86      0.92      0.89      7558
       B-gpe       0.97      0.94      0.96      3142
       B-nat       0.55      0.28      0.37        40
       B-org       0.81      0.73      0.77      4151
       B-per       0.85      0.83      0.84      3400
       B-tim       0.93      0.88      0.90      4077
       I-art       0.25      0.07      0.11        84
       I-eve       0.48      0.23      0.31        65
       I-geo       0.81      0.80      0.81      1462
       I-gpe       0.94      0.52      0.67        33
       I-nat       0.75      0.23      0.35        13
       I-org       0.82      0.80      0.81      3394
       I-per       0.85      0.90      0.87      3406
       I-tim       0.84      0.81      0.82      1251
           O       0.99      0.99      0.99    177590

    accuracy              

In machine learning, precision measures the accuracy of positive predictions (how many predicted positives were actually positive), recall measures the ability to identify all relevant positive cases (how many actual positives were correctly identified), accuracy represents the overall proportion of correct predictions, L1 score calculates the absolute difference between predicted and true values, and support refers to the number of data points belonging to a specific class in a dataset; essentially, how many instances of a particular class are present for evaluation. 

https://www.kaggle.com/code/bavalpreet26/ner-using-crf

Explanation of the CRF Code:

Data Loading and Preprocessing: The load_data and preprocess_data functions are the same as in the HMM example.

Feature Extraction:

word2features(sent, i): This is the heart of the CRF model. It defines the features to be used for each word in the sentence. The function takes the sentence (sent) and the index of the word (i) as input and returns a dictionary of features. The features are designed to capture information about the word itself and its context. This function is critical for CRF performance.

sent2features(sent): Applies word2features to each word in the sentence to create a list of feature dictionaries for the entire sentence.

sent2labels(sent): Extracts the NER tags (labels) for each word in the sentence.

sent2tokens(sent): Extracts the words (tokens) for each word in the sentence.

Prepare Data for CRF:

prepare_crf_data(sentences): Takes the list of sentences and applies sent2features and sent2labels to create the training data X (features) and y (labels).

Train the CRF Model:

train_crf(X_train, y_train): Creates a CRF object, sets the training parameters (algorithm, regularization parameters c1 and c2, maximum iterations), and trains the model using the training data. all_possible_transitions=True allows the model to learn all possible transitions between NER tags, which can improve performance.

Evaluate the CRF Model:

evaluate_crf(crf, X_test, y_test): Uses the trained CRF model to predict the NER tags for the test data. It then prints a detailed classification report (precision, recall, F1-score) for each NER tag using flat_classification_report. The flat_classification_report is used because the data is in a "flat" format (list of lists). The overall accuracy can be calculated from this report as well.

Example Usage: Demonstrates how to use the trained CRF model to predict tags for a new sentence. The example_features part is crucial for making predictions on new data.

Key Differences between HMM and CRF for NER:

Feature Engineering:

HMM: HMMs primarily rely on the observed sequence of words and the assumption that the current tag depends only on the previous tag (Markov assumption). They don't easily incorporate rich, overlapping features of the words and their context.

CRF: CRFs excel at incorporating a wide range of features. The word2features function in the code demonstrates this. We can include features like:

The word itself (lowercased, original case)

Prefixes and suffixes of the word

Whether the word is uppercase, title case, or a digit

The part-of-speech tag (POS)

Features from neighboring words (previous and next words)

Indicator features for beginning and end of sentence.

This ability to use complex, overlapping features is a major advantage of CRFs over HMMs.

Conditional vs. Generative:

HMM: HMMs are generative models. They model the joint probability of the observed sequence (words) and the hidden states (NER tags), P(words, tags). They learn how words are generated from NER tags.

CRF: CRFs are conditional models. They model the conditional probability of the hidden states (NER tags) given the observed sequence (words), P(tags | words). They directly learn to predict the NER tags given the words, without modeling how the words are generated. This direct approach is often more effective for sequence labeling tasks.

Label Bias Problem:

HMM: HMMs can suffer from the "label bias" problem. States with fewer outgoing transitions tend to be favored, even if other states are more appropriate given the context.

CRF: CRFs are less susceptible to the label bias problem because they model the conditional probability directly.

Training:

HMM: HMM training involves estimating transition, emission, and initial probabilities, often using counting and smoothing techniques.

CRF: CRF training involves learning a set of weights for the features. Common training algorithms include L-BFGS (used in the example), gradient descent, and others.

